<a href="https://colab.research.google.com/github/kettn0504/teacher-irv-site/blob/main/%E8%8B%B1%E6%96%87%E5%AD%B8%E7%BF%92%E4%BA%92%E5%8B%95%E7%B6%B2%E7%AB%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# 安裝建立 API 伺服器需要的套件
!pip install fastapi uvicorn nest_asyncio

In [5]:
import threading
import uvicorn
import nest_asyncio
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List, Optional
import random
import re

# 讓 Colab 可以執行非同步伺服器
nest_asyncio.apply()

app = FastAPI()

# --- 資料模型 (對應前端 Tiptap JSON) ---
class VariableAttrs(BaseModel):
    type: str
    rule: str
    originalText: str

class TiptapNode(BaseModel):
    type: str
    text: Optional[str] = None
    attrs: Optional[VariableAttrs] = None

class PreviewRequest(BaseModel):
    content: List[TiptapNode]

# --- 模擬資料庫 (Mock Vocabulary) ---
MOCK_VOCAB = {
    "verb": {
        "past": ["played", "ate", "studied", "wrote", "visited"],
        "base": ["play", "eat", "study", "write", "visit"],
        "pp": ["played", "eaten", "studied", "written", "visited"]
    },
    "noun": {
        "singular": ["piano", "apple", "book", "letter", "museum"],
        "plural": ["pianos", "apples", "books", "letters", "museums"]
    }
}

# --- 核心邏輯 (The Brain) ---
def generate_sentence_from_template(template_str: str) -> str:
    # 替換邏輯：找到 {verb:past} 這種標籤並去查表
    def replacer(match):
        key = match.group(1) # e.g., "verb:past"
        try:
            pos, form = key.split(":")
            if pos in MOCK_VOCAB and form in MOCK_VOCAB[pos]:
                return random.choice(MOCK_VOCAB[pos][form])
            return f"[{key}?]" # 查無此字
        except:
            return key

    return re.sub(r"\{(.*?)\}", replacer, template_str)

def parse_tiptap_to_template(nodes: List[TiptapNode]) -> str:
    # 把 JSON 物件轉回字串模板
    parts = []
    for node in nodes:
        if node.type == "text":
            parts.append(node.text)
        elif node.type == "variableBlock":
            tag = f"{{{node.attrs.type}:{node.attrs.rule}}}"
            parts.append(tag)
    return "".join(parts)

# --- API 接口 ---
@app.post("/api/preview-template")
async def preview_template(request: PreviewRequest):
    raw_template = parse_tiptap_to_template(request.content)
    # 生成 5 個隨機結果
    results = [generate_sentence_from_template(raw_template) for _ in range(5)]
    return {
        "status": "success",
        "parsed_template": raw_template,
        "generated_sentences": results
    }

# --- 在背景啟動伺服器 ---
def run_server():
    print("🚀 伺服器正在背景啟動中...")
    uvicorn.run(app, host="127.0.0.1", port=8000, log_level="warning")

thread = threading.Thread(target=run_server)
thread.start()
print("✅ 伺服器已就緒！可以開始測試 API 了。")

🚀 伺服器正在背景啟動中...✅ 伺服器已就緒！可以開始測試 API 了。



In [6]:
import requests
import json
import time

# 確保伺服器已經完全啟動
time.sleep(1)

print("正在傳送測試資料 (模擬使用者輸入: If I [played] the [piano]...)")
print("-" * 50)

# 這是模擬前端 (React/Tiptap) 傳過來的 JSON
payload = {
    "content": [
        {"type": "text", "text": "If I "},
        # 這裡模擬使用者把 "played" 變成了變數
        {
            "type": "variableBlock",
            "attrs": {"type": "verb", "rule": "past", "originalText": "played"}
        },
        {"type": "text", "text": " the "},
        # 這裡模擬使用者把 "piano" 變成了變數
        {
            "type": "variableBlock",
            "attrs": {"type": "noun", "rule": "singular", "originalText": "piano"}
        },
        {"type": "text", "text": "."}
    ]
}

try:
    # 發送 POST 請求給本地伺服器
    response = requests.post("http://127.0.0.1:8000/api/preview-template", json=payload)
    data = response.json()

    # 顯示結果
    print(f"解析後的模板邏輯: {data['parsed_template']}")
    print("\n✨ 生成的隨機題目 (Random Output):")
    for idx, sentence in enumerate(data['generated_sentences'], 1):
        print(f"{idx}. {sentence}")

except Exception as e:
    print(f"發生錯誤: {e}")

正在傳送測試資料 (模擬使用者輸入: If I [played] the [piano]...)
--------------------------------------------------
解析後的模板邏輯: If I {verb:past} the {noun:singular}.

✨ 生成的隨機題目 (Random Output):
1. If I wrote the apple.
2. If I visited the book.
3. If I studied the book.
4. If I visited the book.
5. If I played the museum.


In [7]:
# ==========================================
# 升級版：加入語意標籤 (Semantic Tags)
# ==========================================

# 1. 資料庫擴充：加入 tags 欄位
MOCK_VOCAB_V2 = {
    "verb": [
        # 食物類動詞
        {"base": "eat", "past": "ate", "tags": ["food_action"]},
        {"base": "cook", "past": "cooked", "tags": ["food_action"]},
        # 音樂類動詞
        {"base": "play", "past": "played", "tags": ["music_action"]},
        {"base": "listen to", "past": "listened to", "tags": ["music_action"]},
        # 閱讀類動詞
        {"base": "read", "past": "read", "tags": ["text_action"]},
        {"base": "write", "past": "wrote", "tags": ["text_action"]},
    ],
    "noun": [
        # 食物類名詞
        {"singular": "apple", "tags": ["food"]},
        {"singular": "pizza", "tags": ["food"]},
        {"singular": "steak", "tags": ["food"]},
        # 音樂類名詞
        {"singular": "piano", "tags": ["instrument"]},
        {"singular": "song", "tags": ["music"]},
        {"singular": "guitar", "tags": ["instrument"]},
        # 閱讀類名詞
        {"singular": "book", "tags": ["text"]},
        {"singular": "letter", "tags": ["text"]},
    ]
}

# 2. 定義邏輯對應規則 (Mapping)
# 告訴系統：如果你選了 "food_action" 的動詞，受詞最好是 "food"
LOGIC_MAP = {
    "food_action": ["food"],
    "music_action": ["instrument", "music"],
    "text_action": ["text"]
}

# 3. 升級版生成器
def generate_smart_sentence(template_str: str):
    # 用來暫存上一輪選到的動詞標籤，以便決定名詞要選什麼
    context = {"verb_tag": None}

    def replacer(match):
        key = match.group(1) # e.g. "verb:past" or "noun:singular"
        parts = key.split(":")
        pos, form = parts[0], parts[1]

        candidates = []

        # --- 處理動詞 ---
        if pos == "verb":
            # 隨機選一個動詞
            word_obj = random.choice(MOCK_VOCAB_V2["verb"])
            # 記錄這個動詞的標籤 (例如 food_action)，給下一個名詞參考
            context["verb_tag"] = word_obj["tags"][0]
            return word_obj[form]

        # --- 處理名詞 ---
        elif pos == "noun":
            # 檢查動詞決定了什麼情境
            required_tags = LOGIC_MAP.get(context["verb_tag"], [])

            # 從名詞庫篩選：必須包含對應標籤的字
            # 如果 required_tags 是空 (沒限制)，就全部名詞都能選
            for noun in MOCK_VOCAB_V2["noun"]:
                if not required_tags or any(tag in noun["tags"] for tag in required_tags):
                    candidates.append(noun)

            if candidates:
                return random.choice(candidates)[form]
            return "[No Logic Match]"

        return key

    return re.sub(r"\{(.*?)\}", replacer, template_str)

# 4. 測試執行
print("--- 升級版：智慧語意生成 ---")
template = "If I {verb:past} the {noun:singular}."

for i in range(5):
    print(f"{i+1}. {generate_smart_sentence(template)}")

--- 升級版：智慧語意生成 ---
1. If I ate the steak.
2. If I cooked the steak.
3. If I cooked the steak.
4. If I read the book.
5. If I cooked the pizza.


In [8]:
# 1. 安裝 NLP 核心套件
!pip install nltk lemminflect

# 2. 下載必要的語料庫資料
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4') # Open Multilingual Wordnet
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

print("✅ 環境建置完成！")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 16.5 MB/s eta 0:00:00


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...


✅ 環境建置完成！


[nltk_data]   Unzipping taggers/universal_tagset.zip.


In [9]:
import lemminflect
from nltk.corpus import wordnet as wn

# --- 工人 A: 型態變化專家 ---
def get_inflections(word, pos):
    """
    輸入: word='eat', pos='VERB'
    輸出: {'past': 'ate', 'pp': 'eaten', ...}
    """
    inflections = {}

    if pos == 'VERB':
        # 取得過去式 (VBD) 和 過去分詞 (VBN)
        inflections['base'] = word
        inflections['past'] = lemminflect.getInflection(word, tag='VBD')[0]
        inflections['pp'] = lemminflect.getInflection(word, tag='VBN')[0]
        inflections['3rd_person'] = lemminflect.getInflection(word, tag='VBZ')[0]
        inflections['ing'] = lemminflect.getInflection(word, tag='VBG')[0]

    elif pos == 'NOUN':
        # 取得複數 (NNS)
        inflections['singular'] = word
        inflections['plural'] = lemminflect.getInflection(word, tag='NNS')[0]

    return inflections

# --- 工人 B: 語意分類專家 (使用 WordNet) ---
def get_auto_tags(word, pos):
    """
    追蹤單字的「祖宗十八代」，找出它是什麼類別。
    例如: Apple -> Edible Fruit -> Fruit -> Food ... -> Entity
    """
    tags = set()

    # 轉換 POS 格式給 WordNet 用 (n=noun, v=verb)
    wn_pos = wn.VERB if pos == 'VERB' else wn.NOUN

    # 取得該單字的第一個定義 (通常是最常用的意思)
    synsets = wn.synsets(word, pos=wn_pos)
    if not synsets:
        return list(tags)

    synset = synsets[0] # 取第一個義項

    # --- 定義我們感興趣的「上位詞 (Hypernyms)」 ---
    # 這些是我們想自動標記的分類
    target_categories = {
        "food": ["food", "meal", "nutrient"],
        "animal": ["animal", "creature"],
        "place": ["location", "area", "structure", "building"],
        "music": ["music", "instrument"],
        "movement": ["travel", "move", "motion"]
    }

    # 遞迴尋找這個字的所有上位詞
    # hypernym_paths() 會回傳這個字到根節點的所有路徑
    for path in synset.hypernym_paths():
        for node in path:
            node_name = node.name().split('.')[0] # e.g., 'food.n.02' -> 'food'

            # 檢查這個節點是否在我們的目標分類中
            for tag, keywords in target_categories.items():
                if node_name in keywords:
                    tags.add(tag)

    return list(tags)

# --- 主產線: 批量生產 ---
def process_vocabulary_list(raw_list):
    processed_data = []

    print(f"🏭 開始加工 {len(raw_list)} 個單字...")
    print("-" * 40)

    for item in raw_list:
        word = item["word"]
        pos = item["pos"] # 'VERB' or 'NOUN'

        # 1. 取得變化形
        inflections = get_inflections(word, pos)

        # 2. 自動判斷標籤
        tags = get_auto_tags(word, pos)

        # 3. 組裝成資料庫格式
        entry = {
            "base_word": word,
            "pos": pos.lower(),
            "inflections": inflections,
            "tags": tags,
            "definition": wn.synsets(word)[0].definition() if wn.synsets(word) else ""
        }
        processed_data.append(entry)

    return processed_data

In [10]:
# 這是你準備匯入的「種子清單」 (只要給原型就好)
seed_vocabulary = [
    {"word": "eat", "pos": "VERB"},
    {"word": "fly", "pos": "VERB"},
    {"word": "piano", "pos": "NOUN"},
    {"word": "banana", "pos": "NOUN"},
    {"word": "dog", "pos": "NOUN"},
    {"word": "library", "pos": "NOUN"},
    {"word": "drive", "pos": "VERB"}
]

# 啟動生產線
final_database = process_vocabulary_list(seed_vocabulary)

# 顯示結果
import json
print(json.dumps(final_database, indent=2))

🏭 開始加工 7 個單字...
----------------------------------------
[
  {
    "base_word": "eat",
    "pos": "verb",
    "inflections": {
      "base": "eat",
      "past": "ate",
      "pp": "eaten",
      "3rd_person": "eats",
      "ing": "eating"
    },
    "tags": [],
    "definition": "take in solid food"
  },
  {
    "base_word": "fly",
    "pos": "verb",
    "inflections": {
      "base": "fly",
      "past": "flew",
      "pp": "flown",
      "3rd_person": "flies",
      "ing": "flying"
    },
    "tags": [
      "movement"
    ],
    "definition": "two-winged insects characterized by active flight"
  },
  {
    "base_word": "piano",
    "pos": "noun",
    "inflections": {
      "singular": "piano",
      "plural": "pianos"
    },
    "tags": [],
    "definition": "a keyboard instrument that is played by depressing keys that cause hammers to strike tuned strings and produce sounds"
  },
  {
    "base_word": "banana",
    "pos": "noun",
    "inflections": {
      "singular": "banana",
   

In [11]:
import random
import re
import lemminflect

# ==========================================
# 1. 定義主詞庫 (Leaders)
# 必須包含人稱 (person) 與單複數 (number)
# ==========================================
SUBJECTS = [
    # 第一人稱
    {"word": "I", "person": 1, "number": "singular", "tags": ["human"]},
    {"word": "We", "person": 1, "number": "plural", "tags": ["human"]},
    # 第二人稱
    {"word": "You", "person": 2, "number": "plural", "tags": ["human"]}, # You 視為複數型態處理
    # 第三人稱單數
    {"word": "He", "person": 3, "number": "singular", "tags": ["human"]},
    {"word": "She", "person": 3, "number": "singular", "tags": ["human"]},
    {"word": "The doctor", "person": 3, "number": "singular", "tags": ["human", "job"]},
    {"word": "The dog", "person": 3, "number": "singular", "tags": ["animal"]},
    # 第三人稱複數
    {"word": "They", "person": 3, "number": "plural", "tags": ["human"]},
    {"word": "The students", "person": 3, "number": "plural", "tags": ["human"]},
]

# ==========================================
# 2. 定義動詞庫 (Followers)
# ==========================================
VERBS = [
    {"base": "eat", "tags": ["food_action"]},
    {"base": "play", "tags": ["game_action", "music_action"]},
    {"base": "study", "tags": ["study_action"]},
    {"base": "teach", "tags": ["study_action"]},
    {"base": "watch", "tags": ["visual_action"]},
    {"base": "be", "tags": ["state"]}, # Be 動詞是魔王，要特別處理
]

# ==========================================
# 3. 文法警察核心 (Grammar Logic)
# ==========================================
def conjugate_verb(verb_base, subject_props, tense):
    """
    根據主詞屬性和時態，決定動詞的正確變化
    """
    person = subject_props["person"]
    number = subject_props["number"]

    # --- 處理 BE 動詞 (最麻煩的特例) ---
    if verb_base == "be":
        if tense == "past":
            # 過去式: was (1st, 3rd sg) / were (others)
            if number == "singular" and person in [1, 3]:
                return "was"
            return "were"
        else: # present
            # 現在式: am, is, are
            if person == 1 and number == "singular": return "am"
            if person == 3 and number == "singular": return "is"
            return "are"

    # --- 處理一般動詞 ---
    if tense == "past":
        # 過去式最簡單，大家都一樣 (VBD)
        return lemminflect.getInflection(verb_base, tag='VBD')[0]

    elif tense == "present":
        # 現在式關鍵判斷：是不是第三人稱單數 (3rd + Singular)?
        if person == 3 and number == "singular":
            # 是 -> 用 VBZ (eats, goes)
            return lemminflect.getInflection(verb_base, tag='VBZ')[0]
        else:
            # 否 -> 用 VBP (eat, go)
            return lemminflect.getInflection(verb_base, tag='VBP')[0]

    return verb_base

# ==========================================
# 4. 具備文法意識的生成器
# ==========================================
def generate_grammatically_correct_sentence(template_str):
    # 用來記住這一句話的主詞是誰
    context = {"subject": None}

    def replacer(match):
        key = match.group(1) # e.g. "subj" or "verb:present"

        # --- 處理主詞 (Subject) ---
        if key.startswith("subj"):
            # 隨機選一個主詞
            subj_obj = random.choice(SUBJECTS)
            # 存入 context，讓後面的動詞可以參考
            context["subject"] = subj_obj
            return subj_obj["word"]

        # --- 處理動詞 (Verb) ---
        elif key.startswith("verb"):
            parts = key.split(":") # ["verb", "present", "food_action"(opt)]
            tense = parts[1] # present or past

            # 1. 確保前面已經有生成主詞了
            if context["subject"] is None:
                return "[Error: Verb before Subject]"

            # 2. 隨機選一個動詞 (這裡省略了 Tag 過濾，先專注文法)
            verb_obj = random.choice(VERBS)

            # 3. *** 呼叫文法警察進行變形 ***
            final_verb = conjugate_verb(verb_obj["base"], context["subject"], tense)

            return final_verb

        return key

    return re.sub(r"\{(.*?)\}", replacer, template_str)

# ==========================================
# 5. 驗證時刻 (Moment of Truth)
# ==========================================

print("--- 測試現在式一致性 (Present Tense Agreement) ---")
template_present = "{subj} {verb:present} English."

for _ in range(5):
    print(generate_grammatically_correct_sentence(template_present))

print("\n--- 測試 BE 動詞過去式 (Past Tense 'Be') ---")
# 我們強制讓動詞變成 be 測試看看
def test_be_sentence():
    subj = random.choice(SUBJECTS)
    verb = conjugate_verb("be", subj, "past")
    return f"{subj['word']} {verb} happy."

for _ in range(5):
    print(test_be_sentence())

--- 測試現在式一致性 (Present Tense Agreement) ---
The students teach English.
You eat English.
We are English.
The doctor watches English.
We eat English.

--- 測試 BE 動詞過去式 (Past Tense 'Be') ---
We were happy.
He was happy.
The doctor was happy.
He was happy.
The students were happy.


In [13]:
import random
import lemminflect

# ==========================================
# 1. 資料庫準備
# ==========================================

# 主詞庫 (原本的動作執行者)
AGENTS = [
    {"word": "The teacher", "person": 3, "number": "singular", "type": "noun"},
    {"word": "The students", "person": 3, "number": "plural", "type": "noun"},
    {"word": "I", "person": 1, "number": "singular", "type": "pronoun"}, # 代名詞需要變格 (I -> me)
    {"word": "He", "person": 3, "number": "singular", "type": "pronoun"}, # 代名詞需要變格 (He -> him)
    {"word": "They", "person": 3, "number": "plural", "type": "pronoun"}
]

# 及物動詞庫 (只有及物動詞才有被動語態！)
# 我們需要 Vpp (過去分詞)
TRANSITIVE_VERBS = [
    {"base": "write", "vpp": "written"},
    {"base": "eat", "vpp": "eaten"},
    {"base": "clean", "vpp": "cleaned"},
    {"base": "design", "vpp": "designed"},
    {"base": "love", "vpp": "loved"},
]

# 受詞庫 (原本的接受者，將變成新主詞)
TARGETS = [
    {"word": "the book", "person": 3, "number": "singular"},
    {"word": "the letters", "person": 3, "number": "plural"},
    {"word": "the apple", "person": 3, "number": "singular"},
    {"word": "the cookies", "person": 3, "number": "plural"},
    {"word": "the website", "person": 3, "number": "singular"},
]

# 代名詞受格對照表 (Subject -> Object)
PRONOUN_OBJ_MAP = {
    "I": "me",
    "He": "him",
    "She": "her",
    "We": "us",
    "They": "them",
    "You": "you"
}

# ==========================================
# 2. 輔助函式：Be 動詞變化計算機
# ==========================================
def get_be_verb(subject_props, tense):
    """
    被動語態的核心：Be 動詞必須跟著「新主詞」變
    """
    person = subject_props["person"]
    number = subject_props["number"]

    if tense == "past":
        # Past: was / were
        if number == "singular" and person in [1, 3]:
            return "was"
        return "were"
    elif tense == "present":
        # Present: am / is / are
        if person == 1 and number == "singular": return "am"
        if person == 3 and number == "singular": return "is"
        return "are"
    elif tense == "future":
        return "will be"

    return "is"

# ==========================================
# 3. 核心邏輯：被動語態生成器
# ==========================================
def generate_passive_sentence(tense="past"):
    # 1. 隨機挑選三個元素
    agent = random.choice(AGENTS)         # 兇手 (原本的主詞)
    verb_obj = random.choice(TRANSITIVE_VERBS) # 動作
    target = random.choice(TARGETS)       # 受害者 (原本的受詞 -> 新主詞)

    # 2. 處理 [新主詞] (The Target)
    # 句子開頭要大寫
    new_subject_text = target["word"].capitalize()

    # 3. 處理 [動詞片語] (Be + Vpp)
    # 關鍵：Be 動詞要配合「新主詞 (Target)」，而不是舊主詞 (Agent)
    be_verb = get_be_verb(target, tense)
    main_verb_pp = verb_obj["vpp"] # 這裡可以結合 lemminflect 自動查，先用硬編碼示範

    # 4. 處理 [by + Agent]
    # 如果 Agent 是代名詞 (He)，要變成受格 (him)
    agent_text = agent["word"]
    if agent.get("type") == "pronoun":
        agent_text = PRONOUN_OBJ_MAP.get(agent_text, agent_text)

    by_phrase = f"by {agent_text}"

    # 5. 組裝
    return f"{new_subject_text} {be_verb} {main_verb_pp} {by_phrase}."

# ==========================================
# 4. 驗證結果
# ==========================================

print("--- 過去被動式 (Past Passive) ---")
# 結構: The apple [was/were] eaten by [him/the teacher].
for _ in range(5):
    print(generate_passive_sentence(tense="past"))

print("\n--- 現在被動式 (Present Passive) ---")
# 結構: The book [is/are] written by [him/the teacher].
for _ in range(5):
    print(generate_passive_sentence(tense="present"))

print("\n--- 未來被動式 (Future Passive) ---")
# 結構: The book will be written by [him/the teacher].
for _ in range(5):
    print(generate_passive_sentence(tense="future"))

--- 過去被動式 (Past Passive) ---
The cookies were designed by them.
The book was eaten by me.
The website was eaten by The students.
The apple was written by The teacher.
The cookies were eaten by The students.

--- 現在被動式 (Present Passive) ---
The letters are written by them.
The apple is eaten by The students.
The cookies are cleaned by The students.
The apple is designed by The students.
The book is written by The teacher.

--- 未來被動式 (Future Passive) ---
The website will be designed by The students.
The apple will be cleaned by me.
The apple will be cleaned by him.
The letters will be eaten by me.
The website will be eaten by The students.


In [14]:
import random

# ==========================================
# 1. 升級版資料庫：加入分類標籤 (Category)
# ==========================================

# 兇手 (Agent) - 這些通常是人，動作比較通用，暫時不需嚴格限制
AGENTS = [
    {"word": "The teacher", "person": 3, "number": "singular", "type": "noun"},
    {"word": "The students", "person": 3, "number": "plural", "type": "noun"},
    {"word": "I", "person": 1, "number": "singular", "type": "pronoun"},
    {"word": "He", "person": 3, "number": "singular", "type": "pronoun"},
    {"word": "The chef", "person": 3, "number": "singular", "type": "noun"}, # 廚師
    {"word": "The writer", "person": 3, "number": "singular", "type": "noun"} # 作家
]

# 動詞 (Verbs) - 加入 target_req (對受詞的要求)
TRANSITIVE_VERBS = [
    {"base": "write", "vpp": "written", "target_req": "text"},   # 寫 -> 需要文字類
    {"base": "read", "vpp": "read", "target_req": "text"},       # 讀 -> 需要文字類
    {"base": "eat", "vpp": "eaten", "target_req": "food"},       # 吃 -> 需要食物類
    {"base": "cook", "vpp": "cooked", "target_req": "food"},     # 煮 -> 需要食物類
    {"base": "design", "vpp": "designed", "target_req": "project"}, # 設計 -> 需要專案類
    {"base": "build", "vpp": "built", "target_req": "project"},     # 建造 -> 需要專案類
]

# 受詞 (Targets/New Subjects) - 加入 category (分類)
TARGETS = [
    # Text 類
    {"word": "the book", "person": 3, "number": "singular", "category": "text"},
    {"word": "the letters", "person": 3, "number": "plural", "category": "text"},
    {"word": "the email", "person": 3, "number": "singular", "category": "text"},
    # Food 類
    {"word": "the apple", "person": 3, "number": "singular", "category": "food"},
    {"word": "the cookies", "person": 3, "number": "plural", "category": "food"},
    {"word": "the steak", "person": 3, "number": "singular", "category": "food"},
    # Project 類
    {"word": "the website", "person": 3, "number": "singular", "category": "project"},
    {"word": "the bridge", "person": 3, "number": "singular", "category": "project"}
]

# 代名詞轉換表
PRONOUN_OBJ_MAP = {"I": "me", "He": "him", "She": "her", "We": "us", "They": "them", "You": "you"}

# ==========================================
# 2. 核心邏輯：加入過濾機制
# ==========================================

def get_be_verb(subject_props, tense):
    person = subject_props["person"]
    number = subject_props["number"]
    if tense == "past":
        if number == "singular" and person in [1, 3]: return "was"
        return "were"
    elif tense == "present":
        if person == 1 and number == "singular": return "am"
        if person == 3 and number == "singular": return "is"
        return "are"
    return "is"

def generate_smart_passive_sentence(tense="past"):
    # Step 1: 先選動詞 (決定情境)
    verb_obj = random.choice(TRANSITIVE_VERBS)
    required_category = verb_obj["target_req"]

    # Step 2: *** 過濾受詞 (關鍵步驟) ***
    # 只從 TARGETS 裡面挑選 category 符合的字
    valid_targets = [t for t in TARGETS if t["category"] == required_category]

    if not valid_targets:
        return f"[Error: No target found for category '{required_category}']"

    target = random.choice(valid_targets)

    # Step 3: 選兇手 (Agent)
    agent = random.choice(AGENTS)

    # Step 4: 組裝句子 (跟之前一樣)
    # 處理新主詞
    new_subject_text = target["word"].capitalize()

    # 處理 Be 動詞 (配合新主詞)
    be_verb = get_be_verb(target, tense)
    main_verb_pp = verb_obj["vpp"]

    # 處理 By + Agent
    agent_text = agent["word"]
    if agent.get("type") == "pronoun":
        agent_text = PRONOUN_OBJ_MAP.get(agent_text, agent_text)

    return f"{new_subject_text} {be_verb} {main_verb_pp} by {agent_text}."

# ==========================================
# 3. 驗證結果
# ==========================================

print("--- 智慧被動語態 (Logic Checked) ---")
for _ in range(8):
    print(generate_smart_passive_sentence(tense="past"))

--- 智慧被動語態 (Logic Checked) ---
The cookies were eaten by him.
The email was read by him.
The bridge was designed by him.
The apple was cooked by The students.
The email was written by him.
The steak was eaten by The students.
The bridge was built by The writer.
The book was written by me.


In [15]:
import random

# ==========================================
# 0. 沿用上一輪的「邏輯資料庫」
# ==========================================
# 為了程式碼獨立性，這裡再次定義資料 (含分類標籤)

AGENTS = [
    {"word": "The teacher", "person": 3, "number": "singular", "type": "noun"},
    {"word": "The students", "person": 3, "number": "plural", "type": "noun"},
    {"word": "He", "person": 3, "number": "singular", "type": "pronoun"},
    {"word": "They", "person": 3, "number": "plural", "type": "pronoun"},
]

TRANSITIVE_VERBS = [
    {"base": "write", "vpp": "written", "target_req": "text"},
    {"base": "read", "vpp": "read", "target_req": "text"},
    {"base": "eat", "vpp": "eaten", "target_req": "food"},
    {"base": "design", "vpp": "designed", "target_req": "project"},
]

TARGETS = [
    {"word": "the book", "person": 3, "number": "singular", "category": "text"},
    {"word": "the letters", "person": 3, "number": "plural", "category": "text"},
    {"word": "the steak", "person": 3, "number": "singular", "category": "food"},
    {"word": "the website", "person": 3, "number": "singular", "category": "project"},
]

PRONOUN_OBJ_MAP = {"I": "me", "He": "him", "She": "her", "We": "us", "They": "them", "You": "you"}

# ==========================================
# 1. 核心工具：助動詞選擇器 (Do-Support)
# ==========================================
def get_do_aux(subject_props, tense):
    """
    決定要用 Do, Does, 還是 Did
    """
    if tense == "past":
        return "Did"

    # Present Tense logic
    person = subject_props["person"]
    number = subject_props["number"]

    if person == 3 and number == "singular":
        return "Does"
    return "Do"

# ==========================================
# 2. 核心工具：Be 動詞選擇器 (For Passive)
# ==========================================
def get_be_verb(subject_props, tense):
    person = subject_props["person"]
    number = subject_props["number"]
    if tense == "past":
        if number == "singular" and person in [1, 3]: return "Was" # 注意首字大寫
        return "Were"
    elif tense == "present":
        if person == 1 and number == "singular": return "Am"
        if person == 3 and number == "singular": return "Is"
        return "Are"
    return "Is"

# ==========================================
# 3. 模式 A: 主動疑問句生成 (Active Question)
# 結構: [Do/Does/Did] + [Subj] + [Verb Base] + [Obj]?
# ==========================================
def generate_active_question(tense="past"):
    # 1. 選動詞 & 根據語意選受詞
    verb_obj = random.choice(TRANSITIVE_VERBS)
    required_category = verb_obj["target_req"]
    valid_targets = [t for t in TARGETS if t["category"] == required_category]
    target = random.choice(valid_targets)

    # 2. 選主詞
    agent = random.choice(AGENTS)

    # 3. *** 關鍵邏輯：選助動詞 ***
    aux = get_do_aux(agent, tense)

    # 4. 處理大小寫與格式
    # 主詞如果不是代名詞(He/They)，放中間要變小寫 (The teacher -> the teacher)
    agent_text = agent["word"]
    if agent["type"] == "noun":
        agent_text = agent_text.lower()
    else:
        # 代名詞如 He, They 放中間要變小寫 (除非是 I)
        agent_text = agent_text.lower()

    # 5. *** 關鍵邏輯：動詞必須用原型 (Base Form) ***
    # 因為前面有 Do/Did 了，後面動詞一律不變化
    main_verb = verb_obj["base"]

    return f"{aux} {agent_text} {main_verb} {target['word']}?"

# ==========================================
# 4. 模式 B: 被動疑問句生成 (Passive Question)
# 結構: [Be] + [New Subj] + [Vpp] + [by Agent]?
# ==========================================
def generate_passive_question(tense="past"):
    # 1. 也是先選字 (邏輯同上)
    verb_obj = random.choice(TRANSITIVE_VERBS)
    required_category = verb_obj["target_req"]
    valid_targets = [t for t in TARGETS if t["category"] == required_category]
    target = random.choice(valid_targets) # 這是新主詞
    agent = random.choice(AGENTS)

    # 2. *** 關鍵邏輯：選 Be 動詞 (移至句首) ***
    # Be 動詞要看「新主詞 (target)」的臉色
    be_verb = get_be_verb(target, tense)

    # 3. 處理中間的名詞 (變小寫)
    # 這裡的 target 是普通名詞 (the book)，要變小寫
    target_text = target["word"].lower() # 其實原本就是小寫，但保險起見

    # 4. 處理 by Agent
    agent_text = agent["word"]
    if agent.get("type") == "pronoun":
        agent_text = PRONOUN_OBJ_MAP.get(agent_text, agent_text)

    return f"{be_verb} {target_text} {verb_obj['vpp']} by {agent_text}?"

# ==========================================
# 5. 驗證結果
# ==========================================

print("--- 主動疑問句 (Active Questions) ---")
print("[Logic: Did/Does + Subj + Base Verb]")
for _ in range(4):
    print(generate_active_question(tense="past")) # 過去式
    print(generate_active_question(tense="present")) # 現在式

print("\n--- 被動疑問句 (Passive Questions) ---")
print("[Logic: Was/Is + Subj + Vpp]")
for _ in range(4):
    print(generate_passive_question(tense="past"))
    print(generate_passive_question(tense="present"))

--- 主動疑問句 (Active Questions) ---
[Logic: Did/Does + Subj + Base Verb]
Did he design the website?
Does he design the website?
Did he design the website?
Do they eat the steak?
Did the teacher write the letters?
Do the students read the letters?
Did the teacher read the book?
Does the teacher write the book?

--- 被動疑問句 (Passive Questions) ---
[Logic: Was/Is + Subj + Vpp]
Were the letters read by The students?
Is the steak eaten by The students?
Were the letters written by The students?
Are the letters written by them?
Was the book written by The students?
Are the letters read by them?
Was the steak eaten by The teacher?
Are the letters written by him?


In [16]:
import random

# ==========================================
# 1. 資料庫 (稍微擴充以測試專有名詞)
# ==========================================
AGENTS = [
    # 普通名詞 (需要變小寫)
    {"word": "The teacher", "person": 3, "number": "singular", "type": "noun"},
    {"word": "The students", "person": 3, "number": "plural", "type": "noun"},
    # 代名詞 (需要變受格或小寫)
    {"word": "He", "person": 3, "number": "singular", "type": "pronoun"},
    {"word": "They", "person": 3, "number": "plural", "type": "pronoun"},
    # 專有名詞 (永遠大寫，測試用)
    {"word": "John", "person": 3, "number": "singular", "type": "proper_noun"},
]

TRANSITIVE_VERBS = [
    {"base": "write", "vpp": "written", "target_req": "text"},
    {"base": "eat", "vpp": "eaten", "target_req": "food"},
    {"base": "design", "vpp": "designed", "target_req": "project"},
]

TARGETS = [
    {"word": "the book", "person": 3, "number": "singular", "category": "text"},
    {"word": "the letters", "person": 3, "number": "plural", "category": "text"},
    {"word": "the steak", "person": 3, "number": "singular", "category": "food"},
    {"word": "the website", "person": 3, "number": "singular", "category": "project"},
]

PRONOUN_OBJ_MAP = {"I": "me", "He": "him", "She": "her", "We": "us", "They": "them", "You": "you"}

# ==========================================
# 2. 核心工具：格式化大師 (New!)
# ==========================================
def format_mid_sentence(word_obj, case_type="subject"):
    """
    word_obj: 資料庫裡那一整包 (包含 word, type)
    case_type: "subject" (主格, ex: Did he...) 或 "object" (受格, ex: by him)
    """
    text = word_obj["word"]
    w_type = word_obj.get("type", "noun")

    # --- 狀況 A: 它是代名詞 (He, They, I) ---
    if w_type == "pronoun":
        if case_type == "object":
            # 轉受格 (He -> him, They -> them)
            # 受格通常已經是小寫了 (him, them, me)
            return PRONOUN_OBJ_MAP.get(text, text)
        else:
            # 維持主格但變小寫 (Did he...?)
            # 例外： "I" 永遠大寫
            if text == "I":
                return "I"
            return text.lower() # He -> he

    # --- 狀況 B: 它是普通名詞 (The teacher) ---
    elif w_type == "noun":
        # 強制轉小寫 (The teacher -> the teacher)
        return text.lower()

    # --- 狀況 C: 它是專有名詞 (John, Taiwan) ---
    elif w_type == "proper_noun":
        # 保持原樣，不動 (John -> John)
        return text

    return text

# ==========================================
# 3. 輔助函式 (沿用)
# ==========================================
def get_be_verb(subject_props, tense):
    person = subject_props["person"]
    number = subject_props["number"]
    if tense == "past":
        if number == "singular" and person in [1, 3]: return "Was"
        return "Were"
    elif tense == "present":
        if person == 1 and number == "singular": return "Am"
        if person == 3 and number == "singular": return "Is"
        return "Are"
    return "Is"

def get_do_aux(subject_props, tense):
    if tense == "past": return "Did"
    person = subject_props["person"]
    number = subject_props["number"]
    if person == 3 and number == "singular": return "Does"
    return "Do"

# ==========================================
# 4. 生成邏輯：套用格式化工具
# ==========================================

def generate_passive_question_v2(tense="past"):
    # 選字
    verb_obj = random.choice(TRANSITIVE_VERBS)
    req = verb_obj["target_req"]
    valid_targets = [t for t in TARGETS if t["category"] == req]
    target = random.choice(valid_targets)
    agent = random.choice(AGENTS)

    # 組合
    be_verb = get_be_verb(target, tense)

    # target 放在中間，要變小寫 (The book -> the book)
    target_text = target["word"].lower()

    # *** 修正點：使用 format_mid_sentence 處理 Agent ***
    # 這裡是 by + Agent，所以是用 "object" (受格) 模式
    agent_text = format_mid_sentence(agent, case_type="object")

    return f"{be_verb} {target_text} {verb_obj['vpp']} by {agent_text}?"

def generate_active_question_v2(tense="past"):
    # 選字
    verb_obj = random.choice(TRANSITIVE_VERBS)
    req = verb_obj["target_req"]
    valid_targets = [t for t in TARGETS if t["category"] == req]
    target = random.choice(valid_targets)
    agent = random.choice(AGENTS)

    # 組合
    aux = get_do_aux(agent, tense)

    # *** 修正點：使用 format_mid_sentence 處理 Agent ***
    # 這裡是 Did + Agent，所以是用 "subject" (主格) 模式
    agent_text = format_mid_sentence(agent, case_type="subject")

    main_verb = verb_obj["base"]

    return f"{aux} {agent_text} {main_verb} {target['word']}?"

# ==========================================
# 5. 驗證時刻
# ==========================================

print("--- 修正版：被動疑問句 (Passive Question) ---")
for _ in range(5):
    print(generate_passive_question_v2(tense="past"))

print("\n--- 修正版：主動疑問句 (Active Question) ---")
for _ in range(5):
    print(generate_active_question_v2(tense="past"))

--- 修正版：被動疑問句 (Passive Question) ---
Was the website designed by the teacher?
Were the letters written by the teacher?
Was the website designed by the teacher?
Was the book written by the students?
Was the book written by John?

--- 修正版：主動疑問句 (Active Question) ---
Did he design the website?
Did they write the letters?
Did he eat the steak?
Did John design the website?
Did he design the website?


In [17]:
import random
import lemminflect # 這是生成錯誤選項的神器

# ==========================================
# 0. 資料庫準備 (沿用上一輪)
# ==========================================
AGENTS = [
    {"word": "The teacher", "person": 3, "number": "singular", "type": "noun"},
    {"word": "The students", "person": 3, "number": "plural", "type": "noun"},
    {"word": "He", "person": 3, "number": "singular", "type": "pronoun"},
    {"word": "They", "person": 3, "number": "plural", "type": "pronoun"},
]

TRANSITIVE_VERBS = [
    {"base": "write", "vpp": "written", "target_req": "text"},
    {"base": "eat", "vpp": "eaten", "target_req": "food"},
    {"base": "design", "vpp": "designed", "target_req": "project"},
    {"base": "buy", "vpp": "bought", "target_req": "food"}, # 新增
]

TARGETS = [
    {"word": "the book", "person": 3, "number": "singular", "category": "text"},
    {"word": "the letters", "person": 3, "number": "plural", "category": "text"},
    {"word": "the steak", "person": 3, "number": "singular", "category": "food"},
    {"word": "the website", "person": 3, "number": "singular", "category": "project"},
]

PRONOUN_OBJ_MAP = {"I": "me", "He": "him", "She": "her", "We": "us", "They": "them", "You": "you"}

# ==========================================
# 1. 核心工具：干擾項生成器 (Distractor Generator)
# ==========================================
def generate_verb_options(base_word, correct_form):
    """
    輸入: base='eat', correct='ate'
    輸出: ['eat', 'eaten', 'eating', 'ate'] (打亂順序)
    """
    options = set()
    options.add(correct_form) # 正解

    # 利用 lemminflect 產生其他變化形當作誘答
    options.add(base_word) # 原型 (eat)
    options.add(lemminflect.getInflection(base_word, tag='VBG')[0]) # 現在分詞 (eating)
    options.add(lemminflect.getInflection(base_word, tag='VBN')[0]) # 過去分詞 (eaten)

    # 為了湊滿4個，如果重複了(例如 put/put/put)，就隨機加 s
    if len(options) < 4:
        options.add(lemminflect.getInflection(base_word, tag='VBZ')[0]) # 第三人稱 (eats)

    final_list = list(options)
    random.shuffle(final_list)
    return final_list

# ==========================================
# 2. 模式 A: 時態填空 (Tense Cloze)
# 題目: He ____ (eat) the steak.
# ==========================================
def generate_tense_cloze():
    # 1. 選材
    verb_obj = random.choice(TRANSITIVE_VERBS)
    req = verb_obj["target_req"]
    target = random.choice([t for t in TARGETS if t["category"] == req])
    agent = random.choice(AGENTS)

    # 2. 決定正確答案 (假設我們考過去式)
    tense = "past"
    correct_verb = lemminflect.getInflection(verb_obj["base"], tag='VBD')[0]

    # 3. *** 挖空處理 ***
    # 我們不直接放 correct_verb，而是放 "____ (base)"
    question_part = f"____ ({verb_obj['base']})"

    # 4. 組裝題目字串
    # 處理主詞格式
    subj_text = agent["word"] # He / The teacher

    sentence = f"{subj_text} {question_part} {target['word']}."

    # 5. 生成選項
    options = generate_verb_options(verb_obj["base"], correct_verb)

    return {
        "type": "Simple Past Tense",
        "question": sentence,
        "answer": correct_verb,
        "options": options
    }

# ==========================================
# 3. 模式 B: 被動語態 Be 動詞填空 (Passive Be Cloze)
# 題目: The letters ____ written by him. (考單複數與時態)
# ==========================================
def generate_passive_be_cloze():
    # 1. 選材
    verb_obj = random.choice(TRANSITIVE_VERBS)
    target = random.choice([t for t in TARGETS if t["category"] == verb_obj["target_req"]])
    agent = random.choice(AGENTS)

    # 2. 決定正確答案 (假設考過去式)
    tense = "past"

    # 判斷 Be 動詞 (依據 Target 的單複數)
    person, number = target["person"], target["number"]
    if number == "singular":
        correct_be = "was"
        distractor_be = "were"
    else:
        correct_be = "were"
        distractor_be = "was"

    # 3. *** 挖空處理 ***
    # 這次提示比較少，考驗文法觀念
    question_part = "____"

    # 4. 組裝
    target_text = target["word"].capitalize() # 句首大寫

    # 處理 Agent (by + 受格)
    agent_text = agent["word"]
    if agent["type"] == "pronoun":
        agent_text = PRONOUN_OBJ_MAP.get(agent_text, agent_text)

    sentence = f"{target_text} {question_part} {verb_obj['vpp']} by {agent_text}."

    # 5. 生成選項 (混淆視聽)
    options = [correct_be, distractor_be, "is", "are"]
    random.shuffle(options)

    return {
        "type": "Passive Voice (Be-Verb Agreement)",
        "question": sentence,
        "answer": correct_be,
        "options": options
    }

# ==========================================
# 4. 驗證與展示
# ==========================================

print("--- 題組 1: 過去式動詞練習 (Verb Tense) ---")
for i in range(1, 4):
    q = generate_tense_cloze()
    print(f"Q{i}: {q['question']}")
    print(f"    [選項]: {q['options']}")
    print(f"    [解答]: {q['answer']}\n")

print("-" * 40)

print("--- 題組 2: 被動語態 Be 動詞練習 (Passive Agreement) ---")
for i in range(1, 4):
    q = generate_passive_be_cloze()
    print(f"Q{i}: {q['question']}")
    print(f"    [選項]: {q['options']}")
    print(f"    [解答]: {q['answer']}\n")

--- 題組 1: 過去式動詞練習 (Verb Tense) ---
Q1: He ____ (buy) the steak.
    [選項]: ['bought', 'buying', 'buy', 'buys']
    [解答]: bought

Q2: He ____ (buy) the steak.
    [選項]: ['buys', 'buying', 'buy', 'bought']
    [解答]: bought

Q3: He ____ (design) the website.
    [選項]: ['design', 'designed', 'designing', 'designs']
    [解答]: designed

----------------------------------------
--- 題組 2: 被動語態 Be 動詞練習 (Passive Agreement) ---
Q1: The book ____ written by him.
    [選項]: ['is', 'are', 'was', 'were']
    [解答]: was

Q2: The steak ____ bought by them.
    [選項]: ['were', 'are', 'was', 'is']
    [解答]: was

Q3: The book ____ written by The students.
    [選項]: ['was', 'are', 'were', 'is']
    [解答]: was



In [19]:
import random

# ==========================================
# 0. 基礎資料庫 (沿用)
# ==========================================
AGENTS = [
    {"word": "The teacher", "person": 3, "number": "singular", "type": "noun"},
    {"word": "The students", "person": 3, "number": "plural", "type": "noun"},
    {"word": "He", "person": 3, "number": "singular", "type": "pronoun"},
    {"word": "They", "person": 3, "number": "plural", "type": "pronoun"},
]

TRANSITIVE_VERBS = [
    {"base": "write", "vpp": "written", "target_req": "text"},
    {"base": "eat", "vpp": "eaten", "target_req": "food"},
    {"base": "design", "vpp": "designed", "target_req": "project"},
    {"base": "buy", "vpp": "bought", "target_req": "food"},
    {"base": "clean", "vpp": "cleaned", "target_req": "place"}, # 新增
]

TARGETS = [
    {"word": "the book", "person": 3, "number": "singular", "category": "text"},
    {"word": "the letters", "person": 3, "number": "plural", "category": "text"},
    {"word": "the steak", "person": 3, "number": "singular", "category": "food"},
    {"word": "the room", "person": 3, "number": "singular", "category": "place"},
    {"word": "the website", "person": 3, "number": "singular", "category": "project"}, # 新增 'project' 類別的目標
]

PRONOUN_OBJ_MAP = {"I": "me", "He": "him", "She": "her", "We": "us", "They": "them", "You": "you"}

# ==========================================
# 1. 新增：時間副詞庫 (Time Anchors)
# ==========================================
TIME_MARKERS = {
    "past": [
        "yesterday", "last night", "two days ago", "last week", "in 2020"
    ],
    "present": [
        "every day", "usually", "on Sundays", "every week", "always"
    ],
    # 未來式暫不使用，因為被動語態未來式結構不同 (will be)
}

# ==========================================
# 2. 升級版：帶時間限制的填空題
# ==========================================
def generate_passive_be_cloze_with_time():
    # 1. 選材
    verb_obj = random.choice(TRANSITIVE_VERBS)

    # 修正點：在選擇 target 之前，先檢查 valid_targets 是否為空
    valid_targets = [t for t in TARGETS if t["category"] == verb_obj["target_req"]]

    if not valid_targets:
        # 如果沒有找到匹配的受詞，返回一個錯誤訊息而不是崩潰
        return {
            "type": "Error",
            "question": f"Error: No matching target found for verb category '{verb_obj['target_req']}'",
            "answer": "N/A",
            "options": []
        }

    target = random.choice(valid_targets)
    agent = random.choice(AGENTS)

    # 2. *** 隨機決定時態 (Past or Present) ***
    tense = random.choice(["past", "present"])

    # 3. 根據時態選擇 Be 動詞 (正確答案)
    person, number = target["person"], target["number"]

    if tense == "past":
        if number == "singular": correct_be = "was"
        else: correct_be = "were"
    else: # present
        if number == "singular": correct_be = "is"
        else: correct_be = "are"

    # 4. *** 關鍵：選取對應的時間詞 ***
    time_marker = random.choice(TIME_MARKERS[tense])

    # 5. 組裝題目
    target_text = target["word"].capitalize()

    agent_text = agent["word"]
    if agent["type"] == "pronoun":
        agent_text = PRONOUN_OBJ_MAP.get(agent_text, agent_text)

    # 格式: [Target] ____ [Vpp] by [Agent] [Time].
    sentence = f"{target_text} ____ {verb_obj['vpp']} by {agent_text} {time_marker}."

    # 6. 生成選項 (一定要包含混淆視聽的錯誤時態)
    # 我們的目標是：既然題目有 yesterday，選項裡一定要有 is/are 來讓學生選錯
    distractors = set()

    # 加入錯誤的數 (Wrong Number)
    if correct_be == "was": distractors.add("were")
    if correct_be == "were": distractors.add("was")
    if correct_be == "is": distractors.add("are")
    if correct_be == "are": distractors.add("is")

    # 加入錯誤的時態 (Wrong Tense) - 這是最重要的干擾項！
    if tense == "past":
        distractors.add("is")
        distractors.add("are")
    else:
        distractors.add("was")
        distractors.add("were")

    # 轉成 list 並洗牌
    final_options = list(distractors)
    # 確保只有 3 個干擾項 + 1 個正確答案
    if len(final_options) > 3:
        final_options = final_options[:3]

    final_options.append(correct_be)
    random.shuffle(final_options)

    return {
        "type": f"Passive Voice ({tense.capitalize()})",
        "question": sentence,
        "answer": correct_be,
        "options": final_options
    }

# ==========================================
# 3. 驗證結果
# ==========================================

print("--- 嚴謹版：附帶時間提示的填空題 ---")
for i in range(1, 6):
    q = generate_passive_be_cloze_with_time()
    print(f"Q{i}: {q['question']}")
    print(f"    [選項]: {q['options']}")
    print(f"    [解答]: {q['answer']}\n")

--- 嚴謹版：附帶時間提示的填空題 ---
Q1: The steak ____ eaten by him last night.
    [選項]: ['is', 'was', 'are', 'were']
    [解答]: was

Q2: Error: No matching target found for verb category 'project'
    [選項]: []
    [解答]: N/A

Q3: The steak ____ bought by The teacher usually.
    [選項]: ['was', 'is', 'are', 'were']
    [解答]: is

Q4: Error: No matching target found for verb category 'project'
    [選項]: []
    [解答]: N/A

Q5: The steak ____ bought by him in 2020.
    [選項]: ['is', 'are', 'were', 'was']
    [解答]: was

